In [1]:
from pymilvus import connections, utility
from pymilvus import Collection, DataType, FieldSchema, CollectionSchema
import time
from os import getenv
from dotenv import load_dotenv, find_dotenv

In [2]:
from helper import *

e:\RAG\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
chunks = text_splitter("output.txt")

Total chunks: 2886


In [4]:
emb = generate_embeddings(chunks[:250])
chu = chunks[:250]

KeyboardInterrupt: 

In [ ]:
#embeddings = generate_embeddings(chunks)

In [ ]:
dimensions = emb[0].shape[0]
dimensions

In [6]:
connect_milvus()

Connecting to Milvus...
Connected to Milvus!


In [ ]:
collection_name = "ChumBucket"

In [ ]:
if utility.has_collection(collection_name):
    drop_result = utility.drop_collection(collection_name)

""" text_field = FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=8192, description="text", auto_id=False, is_primary=True)
embedding_field = FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384, description="embedding", is_primary=False, auto_id=False)

schema = CollectionSchema(fields=[text_field, embedding_field], description="collection description")

print(f"Creating collection: {collection_name}")
collection = Collection(name=collection_name, schema=schema)
print(f"Schema: {schema}")
print("Success!") """

In [ ]:
collection = create_collection(collection_name, dimensions)

In [ ]:
insert_embeddings(collection, chu, emb)

In [ ]:
# flush data
print("Flushing data...")
collection.flush()

print("Data flushed successfully!")

In [ ]:
# create index

print("Creating index...")

index_params = {"index_type": "AUTOINDEX", "metric_type": "L2", "params": {}}

collection.create_index(field_name="embedding", index_params=index_params)

print("Index created successfully!")


In [ ]:
# load data into collection

print("Loading data...")
collection.load()

print("Data loaded successfully!")

In [9]:
emb1 = generate_embeddings(["What is Rstudio?"])

KeyboardInterrupt: 

In [10]:
model = SentenceTransformer("all-MiniLM-L12-v2")

In [11]:
emb1 = model.encode(["What is Rstudio?"])

In [7]:
collection = Collection(name="Chums")

In [12]:
res = semantic_search(collection, emb1, 3)

Searching for similar embeddings...
Search completed!


In [13]:
print(res)

["['id: menu, as shown in Figure 4-10. This book assumes that you use the default Start menu setup. If you choose some other location, you may need to modify procedures later in the book. FIGURE 4-8: The setup process begins by telling you that you’re installing RStudio. FIGURE 4-9: Specify an installation location for RStudio. CHAPTER 4 Installing an R Distribution 5516. Choose Start Menu Configuration (if necessary) and then click Install. You see an Installing dialog box that tells you about the installation progress. After the process completes, you see a completion dialog box appear in its place. 17. Click Finish. You’re ready to begin using RStudio. FIGURE 4-10: Define the Start menu informa- tion as needed. A WORD ABOUT THE SCREENSHOTS As you work your way through the book, you’ll use an IDE of your choice to open the R files containing the book’s source code. Every screenshot that contains IDE-specific information relies on RStudio because RStudio runs on all three platforms su

In [ ]:
# search for similar embeddings

print("Searching for similar embeddings...")
search_params = {"metric_type": "L2", "params": {"nprobe": 16}}
query_res = collection.search(emb1, "embedding", search_params, limit=5, expr=None, output_fields=["text"])

print(query_res[0])

In [ ]:
connections.disconnect("default")